In [484]:
class Plot:
    def __init__(self,x,y, name):
        self.x = x
        self.y = y
        self.name = name
        self.adjacent = []
        self.checked = False
        self.area = None
        self.right = True
        self.bottom = True
        self.left = True
        self.top = True

    @property
    def perimeter(self):
        return 4 - len(self.adjacent)
    
    def __repr__(self):
        return self.name

In [485]:
class Area:
    def __init__(self, name):
        self.plots = []
        self.name = name
        self.x = None
        self.y = None
        self.h = None
        self.w = None
        self.zones = None
        self.sides = []

    def add_plot(self, plot):
        if plot not in self.plots:
            plot.checked = True
            self.plots.append(plot)
            plot.area = self
            
        for adjacent in plot.adjacent:
            if adjacent not in self.plots:
                if not adjacent.checked:
                    adjacent.checked = True
                    self.plots.append(adjacent)
                    plot.area = self
                    self.add_plot(adjacent)

    @property
    def area(self):
        return len(self.plots)
    
    @property
    def perimeter(self):
        return sum([plot.perimeter for plot in self.plots])
    
    @property
    def price(self):
        return self.area * self.perimeter
    
    @property
    def price_discount(self):
        return self.total_sides * self.area
    
    @property
    def total_sides(self):
        if self.x is None:
            self.set_pos()
        # top
        top = 0
        for y in range(self.y, self.y + self.h):
            row = filter(lambda plot: plot.y == y, self.plots)
            row = sorted(row, key=lambda plot: plot.x)

            if len(row) == 1:
                if row[0].top:
                    top += 1
                    continue

            contiguos = False
            for i in range(len(row) - 1):
                plot_1 = row[i]
                plot_2 = row[i + 1]

                if not contiguos and plot_1.top:
                    top += 1
                    contiguos = True

                diff = plot_2.x - plot_1.x
                if diff > 1:
                    contiguos = False

                if not plot_1.top or not plot_2.top:
                    contiguos = False
                
                if not contiguos and plot_2.top:
                    top += 1
                    contiguos = True

        # bottom
        bottom = 0
        for y in range(self.y, self.y + self.h):
            row = filter(lambda plot: plot.y == y, self.plots)
            row = sorted(row, key=lambda plot: plot.x)

            contiguos = False
            if len(row) == 1:
                if row[0].bottom:
                    bottom += 1
                    continue
            for i in range(len(row) - 1):
                plot_1 = row[i]
                plot_2 = row[i + 1]

                if not contiguos and plot_1.bottom:
                    bottom += 1
                    contiguos = True

                diff = plot_2.x - plot_1.x
                if diff > 1:
                    contiguos = False

                if not plot_1.bottom or not plot_2.bottom:
                    contiguos = False
                
                if not contiguos and plot_2.bottom:
                    bottom += 1
                    contiguos = True


        # left
        left = 0
        for x in range(self.x, self.x + self.w):
            column = filter(lambda plot: plot.x == x, self.plots)
            column = sorted(column, key=lambda plot: plot.y)
            contiguos = False
            if len(column) == 1:
                if column[0].left:
                    left += 1
                    continue
            for i in range(len(column) - 1):
                plot_1 = column[i]
                plot_2 = column[i + 1]

                if not contiguos and plot_1.left:
                    left += 1
                    contiguos = True

                diff = plot_2.y - plot_1.y
                if diff > 1:
                    contiguos = False

                if not plot_1.left or not plot_2.left:
                    contiguos = False
                
                if not contiguos and plot_2.left:
                    left += 1
                    contiguos = True


        # right
        right = 0
        for x in range(self.x, self.x + self.w):
            column = filter(lambda plot: plot.x == x, self.plots)
            column = sorted(column, key=lambda plot: plot.y)

            contiguos = False
            if len(column) == 1:
                if column[0].right:
                    right += 1
                    continue
            for i in range(len(column) - 1):
                plot_1 = column[i]
                plot_2 = column[i + 1]

                if not contiguos and plot_1.right:
                    right += 1
                    contiguos = True

                diff = plot_2.y - plot_1.y
                if diff > 1:
                    contiguos = False

                if not plot_1.right or not plot_2.right:
                    contiguos = False
                
                if not contiguos and plot_2.right:
                    right += 1
                    contiguos = True


        self.sides = [top, bottom, left, right]
        return top + bottom + left + right

    def set_pos(self):
        self.x = min([x.x for x in self.plots])
        self.y = min([y.y for y in self.plots])
        self.h = max([y.y for y in self.plots]) - self.y + 1
        self.w = max([x.x for x in self.plots]) - self.x + 1
    
    def __str__(self):
        return f'{self.name} area: {self.area} perimeter: {self.perimeter} price: {self.price} total_sides: {self.total_sides}'

    def __repr__(self):
        return self.name

In [486]:
zone = []

with open('./example_day12.txt') as f:
    for i, line in enumerate(f):
        zone.append([Plot(j, i, name) for j,name in enumerate(line.strip())])

zone

[[R, R, R, R, I, I, C, C, F, F],
 [R, R, R, R, I, I, C, C, C, F],
 [V, V, R, R, R, C, C, F, F, F],
 [V, V, R, C, C, C, J, F, F, F],
 [V, V, V, V, C, J, J, C, F, E],
 [V, V, I, V, C, C, J, J, E, E],
 [V, V, I, I, I, C, J, J, E, E],
 [M, I, I, I, I, I, J, J, E, E],
 [M, I, I, I, S, I, J, E, E, E],
 [M, M, M, I, S, S, J, E, E, E]]

In [487]:
for row in zone:
    for plot in row:
        for i, j in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            ny, nx = plot.y + i, plot.x + j
            if 0 <= ny < len(zone) and 0 <= nx < len(row):  # Verifica límites correctos
                if plot.name == zone[ny][nx].name:
                    plot.adjacent.append(zone[ny][nx])

                    if i == 1:
                        plot.bottom = False
                    elif i == -1:
                        plot.top = False
                    elif j == 1:
                        plot.right = False
                    elif j == -1:
                        plot.left = False

In [488]:
areas = []

for row in zone:
    for plot in row:
        if plot.checked == False:
            plot.checked = True
            area = Area(plot.name)
            area.zones = zone
            area.add_plot(plot)
            areas.append(area)
areas

[R, I, C, F, V, J, C, E, I, M, S]

In [489]:
for area in areas:
    area.total_sides
    print(f"area: {area.name} price {area.area} * {area.total_sides} = {area.price_discount} -- {area.sides}")

area: R price 12 * 10 = 120 -- [2, 3, 2, 3]
area: I price 4 * 4 = 16 -- [1, 1, 1, 1]
area: C price 14 * 22 = 308 -- [5, 6, 5, 6]
area: F price 10 * 12 = 120 -- [2, 4, 4, 2]
area: V price 13 * 10 = 130 -- [2, 3, 2, 3]
area: J price 11 * 12 = 132 -- [3, 3, 3, 3]
area: C price 1 * 4 = 4 -- [1, 1, 1, 1]
area: E price 13 * 8 = 104 -- [3, 1, 3, 1]
area: I price 14 * 16 = 224 -- [4, 4, 4, 4]
area: M price 5 * 6 = 30 -- [2, 1, 1, 2]
area: S price 3 * 6 = 18 -- [2, 1, 1, 2]


In [490]:
price = sum([area.price for area in areas])
price

1930

In [491]:
discount = sum([area.price_discount for area in areas])
discount

1206

# Part 1

In [492]:
zone = []

with open('./data_day12.txt') as f:
    for i, line in enumerate(f):
        zone.append([Plot(j, i, x) for j,x in enumerate(line.strip())])

In [493]:
for row in zone:
    for plot in row:
        for i, j in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            ny, nx = plot.y + i, plot.x + j
            if 0 <= ny < len(zone) and 0 <= nx < len(row):  # Verifica límites correctos
                if plot.name == zone[ny][nx].name:
                    plot.adjacent.append(zone[ny][nx])

                    if i == 1:
                        plot.bottom = False
                    elif i == -1:
                        plot.top = False
                    elif j == 1:
                        plot.right = False
                    elif j == -1:
                        plot.left = False


In [494]:
areas = []

for row in zone:
    for plot in row:
        if plot.checked == False:
            plot.checked = True
            area = Area(plot.name)
            area.add_plot(plot)
            areas.append(area)


In [495]:
price = sum([area.price for area in areas])
price

1433460

In [496]:
discount = sum([area.price_discount for area in areas])
discount

855082